In [1]:
import pandas as pd

athletes = pd.read_csv('athlete_events.csv')
noc = pd.read_csv('noc_regions.csv')

In [2]:
print(f"athlete_events shape: {athletes.shape}")
print(f"noc_regions shape: {noc.shape}")

athlete_events shape: (271116, 15)
noc_regions shape: (230, 3)


In [5]:
print(athletes.head())
print(athletes.info())
print(athletes.describe())

   ID                      Name Sex   Age  Height  Weight            Team  \
0   1                 A Dijiang   M  24.0   180.0    80.0           China   
1   2                  A Lamusi   M  23.0   170.0    60.0           China   
2   3       Gunnar Nielsen Aaby   M  24.0     NaN     NaN         Denmark   
3   4      Edgar Lindenau Aabye   M  34.0     NaN     NaN  Denmark/Sweden   
4   5  Christine Jacoba Aaftink   F  21.0   185.0    82.0     Netherlands   

   NOC        Games  Year  Season       City          Sport  \
0  CHN  1992 Summer  1992  Summer  Barcelona     Basketball   
1  CHN  2012 Summer  2012  Summer     London           Judo   
2  DEN  1920 Summer  1920  Summer  Antwerpen       Football   
3  DEN  1900 Summer  1900  Summer      Paris     Tug-Of-War   
4  NED  1988 Winter  1988  Winter    Calgary  Speed Skating   

                              Event Medal  
0       Basketball Men's Basketball   NaN  
1      Judo Men's Extra-Lightweight   NaN  
2           Football Men's

In [6]:
print(noc.head())
print(noc.tail())
print(noc.info())

   NOC       region                 notes
0  AFG  Afghanistan                   NaN
1  AHO      Curacao  Netherlands Antilles
2  ALB      Albania                   NaN
3  ALG      Algeria                   NaN
4  AND      Andorra                   NaN
     NOC    region        notes
225  YEM     Yemen          NaN
226  YMD     Yemen  South Yemen
227  YUG    Serbia   Yugoslavia
228  ZAM    Zambia          NaN
229  ZIM  Zimbabwe          NaN
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 230 entries, 0 to 229
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   NOC     230 non-null    object
 1   region  227 non-null    object
 2   notes   21 non-null     object
dtypes: object(3)
memory usage: 5.5+ KB
None


In [7]:
print(athletes.isnull().sum())

ID             0
Name           0
Sex            0
Age         9474
Height     60171
Weight     62875
Team           0
NOC            0
Games          0
Year           0
Season         0
City           0
Sport          0
Event          0
Medal     231333
dtype: int64


In [8]:
noc_athletes = set(athletes['NOC'].unique())
noc_master = set(noc['NOC'].unique())

missing_nocs = noc_athletes - noc_master

print(f"Missing NOCs (not in noc_regions.csv): {missing_nocs}")

Missing NOCs (not in noc_regions.csv): {'SGP'}


In [9]:
noc['NOC'] = noc['NOC'].replace({
    'SIN': 'SGP'
})

In [10]:
df = pd.merge(athletes, noc, how='left', on='NOC')
print(df.head())

   ID                      Name Sex   Age  Height  Weight            Team  \
0   1                 A Dijiang   M  24.0   180.0    80.0           China   
1   2                  A Lamusi   M  23.0   170.0    60.0           China   
2   3       Gunnar Nielsen Aaby   M  24.0     NaN     NaN         Denmark   
3   4      Edgar Lindenau Aabye   M  34.0     NaN     NaN  Denmark/Sweden   
4   5  Christine Jacoba Aaftink   F  21.0   185.0    82.0     Netherlands   

   NOC        Games  Year  Season       City          Sport  \
0  CHN  1992 Summer  1992  Summer  Barcelona     Basketball   
1  CHN  2012 Summer  2012  Summer     London           Judo   
2  DEN  1920 Summer  1920  Summer  Antwerpen       Football   
3  DEN  1900 Summer  1900  Summer      Paris     Tug-Of-War   
4  NED  1988 Winter  1988  Winter    Calgary  Speed Skating   

                              Event Medal       region notes  
0       Basketball Men's Basketball   NaN        China   NaN  
1      Judo Men's Extra-Lightwei

In [11]:
missing_rows = df[df['region'].isnull()]
print(f"Rows with blank region: {missing_rows.shape[0]}")

Rows with blank region: 21


In [12]:
df['region'] = df['region'].fillna('Unknown')
print(f"Blank regions after fill: {df['region'].isnull().sum()}")

Blank regions after fill: 0


In [15]:
df['Height_Imputed'] = df.groupby(['Sport', 'Sex'])['Height'].transform(lambda x: x.fillna(x.mean()))
overall_height_mean = df['Height'].mean()
df['Height_Imputed'] = df['Height_Imputed'].fillna(overall_height_mean)

In [16]:
df['Weight_Imputed'] = df.groupby(['Sport', 'Sex'])['Weight'].transform(lambda x: x.fillna(x.mean()))
overall_weight_mean = df['Weight'].mean()
df['Weight_Imputed'] = df['Weight_Imputed'].fillna(overall_weight_mean)

In [17]:
df['Age_Imputed'] = df.groupby(['Sport', 'Sex'])['Age'].transform(lambda x: x.fillna(x.mean()))
overall_age_mean = df['Age'].mean()
df['Age_Imputed'] = df['Age_Imputed'].fillna(overall_age_mean)

In [18]:
df['IsMedalist'] = df['Medal'].notnull().astype(int)

In [19]:
print(df.head())

   ID                      Name Sex   Age  Height  Weight            Team  \
0   1                 A Dijiang   M  24.0   180.0    80.0           China   
1   2                  A Lamusi   M  23.0   170.0    60.0           China   
2   3       Gunnar Nielsen Aaby   M  24.0     NaN     NaN         Denmark   
3   4      Edgar Lindenau Aabye   M  34.0     NaN     NaN  Denmark/Sweden   
4   5  Christine Jacoba Aaftink   F  21.0   185.0    82.0     Netherlands   

   NOC        Games  Year  ...       City          Sport  \
0  CHN  1992 Summer  1992  ...  Barcelona     Basketball   
1  CHN  2012 Summer  2012  ...     London           Judo   
2  DEN  1920 Summer  1920  ...  Antwerpen       Football   
3  DEN  1900 Summer  1900  ...      Paris     Tug-Of-War   
4  NED  1988 Winter  1988  ...    Calgary  Speed Skating   

                              Event Medal       region notes Height_Imputed  \
0       Basketball Men's Basketball   NaN        China   NaN     180.000000   
1      Judo Men's 

In [56]:
medal_summary = df[df['IsMedalist'] == 1].groupby(['region', 'Year'])['Medal'].count().reset_index()
medal_summary.rename(columns={'Medal': 'Total_Medals'}, inplace=True)
print(medal_summary.head())

        region  Year  Total_Medals
0  Afghanistan  2008             1
1  Afghanistan  2012             1
2      Algeria  1984             2
3      Algeria  1992             2
4      Algeria  1996             3


In [21]:
print(df.head())
print(df[['Height_Imputed', 'Weight_Imputed', 'Age_Imputed']].isnull().sum())

   ID                      Name Sex   Age  Height  Weight            Team  \
0   1                 A Dijiang   M  24.0   180.0    80.0           China   
1   2                  A Lamusi   M  23.0   170.0    60.0           China   
2   3       Gunnar Nielsen Aaby   M  24.0     NaN     NaN         Denmark   
3   4      Edgar Lindenau Aabye   M  34.0     NaN     NaN  Denmark/Sweden   
4   5  Christine Jacoba Aaftink   F  21.0   185.0    82.0     Netherlands   

   NOC        Games  Year  ...       City          Sport  \
0  CHN  1992 Summer  1992  ...  Barcelona     Basketball   
1  CHN  2012 Summer  2012  ...     London           Judo   
2  DEN  1920 Summer  1920  ...  Antwerpen       Football   
3  DEN  1900 Summer  1900  ...      Paris     Tug-Of-War   
4  NED  1988 Winter  1988  ...    Calgary  Speed Skating   

                              Event Medal       region notes Height_Imputed  \
0       Basketball Men's Basketball   NaN        China   NaN     180.000000   
1      Judo Men's 

In [22]:
df.to_csv('athletes_clean.csv', index=False)